## Preparación del notebook con OpenAI API key

In [41]:
import sys
import os
import json
import gradio as gr
sys.path.append('../src')
from procesador_de_cvs_con_llm import ProcesadorCV
from dotenv import load_dotenv
load_dotenv("../../../../../../../apis/.env")
api_key = os.getenv("OPENAI_API_KEY")
unmasked_chars = 8
masked_key = api_key[:unmasked_chars] + '*' * (len(api_key) - unmasked_chars*2) + api_key[-unmasked_chars:]
print(f"API key: {masked_key}")

API key: sk-proj-****************************************************************************************************************************************************-amA_5sA


## Prueba del módulo de procesamiento

In [ ]:
# Parámetros de ejecución:
job_text = "Generative AI engineer"
cv_sample_path = '../../ejemplos_cvs/DavidGR_cv.txt' # Ruta al fichero de texto con un currículo de ejemplo
with open(cv_sample_path, 'r') as file:
    cv_text = file.read()
# Prompts:
with open('../prompts/ner_pre_prompt.txt', 'r', encoding='utf-8') as f:
    ner_pre_prompt = f.read()
with open('../prompts/system_prompt.txt', 'r', encoding='utf-8') as f:
    system_prompt = f.read()
with open('../prompts/user_prompt.txt', 'r', encoding='utf-8') as f:
    user_prompt = f.read()
# Esquemas JSON:
with open('../json/ner_schema.json', 'r', encoding='utf-8') as f:
    ner_schema = json.load(f)
with open('../json/response_schema.json', 'r', encoding='utf-8') as f:
    response_schema = json.load(f)


procesador_cvs_prueba_final = ProcesadorCV(api_key, cv_text, job_text, ner_pre_prompt, 
                                           system_prompt, user_prompt, ner_schema, response_schema)
req_experience = 48 # Experiencia requerida en meses
positions_cap=10 # Número máximo de puestos a considerar
dist_threshold_low=0.5 # Distancia límite para considerar un puesto equivalente
dist_threshold_high=0.7 # Distancia límite para considerar un puesto no relevante
dict_respuesta = procesador_cvs_prueba_final.procesar_cv_completo(req_experience=req_experience,
                                                positions_cap=positions_cap,
                                                dist_threshold_low=dist_threshold_low,
                                                dist_threshold_high=dist_threshold_high
                                                )

Cliente inicializado como <openai.OpenAI object at 0x00000168642D44D0>
Respuesta:
 {
    "puntuacion": 100,
    "experiencia": [
        {
            "empresa": "Talking to Chatbots, by Reddgr",
            "puesto": "Web Publisher and Generative AI Researcher",
            "duracion": 218
        },
        {
            "empresa": "IBM",
            "puesto": "Relationship Manager | Cognitive Solutions SaaS",
            "duracion": 43
        },
        {
            "empresa": "Acoustic",
            "puesto": "Principal Consultant | Martech SaaS",
            "duracion": 35
        },
        {
            "empresa": "IBM",
            "puesto": "Engagement Manager | B2B SaaS Retail Analytics",
            "duracion": 9
        },
        {
            "empresa": "IBM",
            "puesto": "Engagement Manager, in support of Acoustic | B2B SaaS Retail Analytics",
            "duracion": 10
        },
        {
            "empresa": "MBD Analytics",
            "puesto": "Busine

## Prueba de la aplicación Gradio

Función de carga de la aplicación de "backend" para la interfaz Gradio:

In [16]:
def process_cv(job_text, cv_text, req_experience, positions_cap, dist_threshold_low, dist_threshold_high):
    if dist_threshold_low >= dist_threshold_high:
        return {"error": "dist_threshold_low debe ser más bajo que dist_threshold_high."}
    
    if not isinstance(cv_text, str) or not cv_text.strip():
        return {"error": "Por favor, introduce el CV o sube un fichero."}

    try:
        procesador = ProcesadorCV(api_key, cv_text, job_text, ner_pre_prompt, 
                                  system_prompt, user_prompt, ner_schema, response_schema)
        dict_respuesta = procesador.procesar_cv_completo(
            req_experience=req_experience,
            positions_cap=positions_cap,
            dist_threshold_low=dist_threshold_low,
            dist_threshold_high=dist_threshold_high
        )
        return dict_respuesta
    except Exception as e:
        return {"error": f"Error en el procesamiento: {str(e)}"}

Interfaz de Gradio:

In [39]:
# Fichero de ejemplo para autocompletar (opción que aparece en la parte de abajo de la interfaz de usuario):
with open('../cv_examples/reddgr_cv.txt', 'r') as file:
    cv_example = file.read()

# Componentes de la aplicación:
job_text_input = gr.Textbox(label="Título oferta de trabajo", lines=1, placeholder="Introduce el título de la oferta de trabajo")
cv_text_input = gr.Textbox(label="CV en formato texto", lines=5, max_lines=5, placeholder="Introduce el texto del CV")
req_experience_input = gr.Number(label="Experiencia requerida (en meses)", value=48, precision=0)
positions_cap_input = gr.Number(label="Número máximo de puestos a extraer", value=10, precision=0)
dist_threshold_low_slider = gr.Slider(label="Umbral mínimo de distancia de embeddings (puesto equivalente)", 
                                      minimum=0, maximum=1, value=0.5, step=0.05)
dist_threshold_high_slider = gr.Slider(label="Umbral máximo de distancia de embeddings (puesto irrelevante)", 
                                       minimum=0, maximum=1, value=0.7, step=0.05)
submit_button = gr.Button("Procesar")

# Código CSS para truncar el texto de ejemplo en la interfaz (bloque "Examples" en la parte de abajo):
css = """
        table tbody tr {
            height: 2.5em; /* Set a fixed height for the rows */
            overflow: hidden; /* Hide overflow content */
        }

        table tbody tr td {
            overflow: hidden; /* Ensure content within cells doesn't overflow */
            text-overflow: ellipsis; /* Add ellipsis for overflowing text */
            white-space: nowrap; /* Prevent text from wrapping */
            vertical-align: middle; /* Align text vertically within the fixed height */
        }
        """

# Interfaz Gradio:
interface = gr.Interface(
    fn=process_cv,
    inputs=[
        job_text_input, 
        cv_text_input, 
        req_experience_input, 
        positions_cap_input, 
        dist_threshold_low_slider, 
        dist_threshold_high_slider
    ],
    outputs="json",
    title="Procesador de CVs con LLM",
    description=("Introduce el título de la oferta de trabajo, el texto del CV, y ajusta los parámetros para analizar la candidatura. "
                 "Puedes utilizar algunos de los ejemplos proporcionados en la parte inferior. Haz click en los ejemplos para autocompletar."),
    examples=[
        ["Cajero de supermercado", "Trabajo de charcutero desde 2021. Antes trabajé 2 meses de camarero en un bar de tapas.", 48, 10, 0.5, 0.7],
        ["Generative AI Engineer", cv_example, 48, 10, 0.5, 0.7]
    ],
    live=False,
    allow_flagging="never",  # Disables the "Flag" button as there is no flag handler yet
    css=css,
    article=(
            "<footer><p>Puedes consultar el código completo de esta app y los notebooks explicativos en "
            "<a href='https://github.com/reddgr' target='_blank'>GitHub</a></p>"
            "<p>© 2024 <a href='https://talkingtochatbots.com' target='_blank'>talkingtochatbots.com</a></p></footer>"
    )
)

# Lanzar la aplicación:
if __name__ == "__main__":
    interface.launch()

Running on local URL:  http://127.0.0.1:7882

To create a public link, set `share=True` in `launch()`.


c:\Users\david\anaconda3\Lib\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(
